In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("./input"))

['data.csv', 'data_orignal.csv']


In [2]:
from fuzzywuzzy import fuzz
import xgboost as xgb
import nltk
from nltk.corpus import stopwords
stopword=nltk.corpus.stopwords.words('english')
import string
string.punctuation

C:\Users\ACER\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [3]:
df=pd.read_csv('./input/data.csv')
print(df.info())
print("data shape : ",df.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 21 columns):
Unnamed: 0               10000 non-null int64
Unnamed: 0.1             10000 non-null int64
id                       10000 non-null int64
qid1                     10000 non-null int64
qid2                     10000 non-null int64
question1                9999 non-null object
question2                10000 non-null object
is_duplicate             10000 non-null int64
wmdistance               10000 non-null float64
len_q1                   10000 non-null int64
len_q2                   10000 non-null int64
diff_len                 10000 non-null int64
len_char_q1              10000 non-null int64
len_char_q2              10000 non-null int64
len_word_q1              10000 non-null int64
len_word_q2              10000 non-null int64
common_words             10000 non-null int64
fuzz_qratio              10000 non-null int64
fuzz_WRatio              10000 non-null int64
fuzz_partial_ra

In [4]:
df.drop(df[df.isnull().any(axis=1)].index,inplace=True)

In [5]:
df.shape

(9999, 21)

In [6]:
fe=['diff_len','common_words','fuzz_qratio','fuzz_WRatio','fuzz_partial_ratio','fuzz_token_sort_ratio']

In [22]:
X=df[fe]
y=df['is_duplicate']

## this classifier has been selected after comparing with many other models
### for more info go to 'model_selection' file

In [9]:
clf=xgb.XGBClassifier(objective ='reg:linear', colsample_bytree = 0.75, learning_rate = 0.089,
                max_depth = 8, alpha = 10, n_estimators = 10)

In [10]:
clf.fit(X,y)

[01:22:36] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBClassifier(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.75, gamma=0,
       learning_rate=0.089, max_delta_step=0, max_depth=8,
       min_child_weight=1, missing=None, n_estimators=10, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

In [51]:
q1=input("first question - ")
q2=input("first question - ")

first question - what is the time rigt now in india?
first question - what time is it in pakistan?


In [52]:
data=[[q1,q2]]
dat=pd.DataFrame(data,columns=['question1','question2'])
dat

,question1,question2
0,what is the time rigt now in india?,what time is it in pakistan?


In [53]:
dat['len_q1'] = dat.question1.apply(lambda x: len(str(x)))
dat['len_q2'] = dat.question2.apply(lambda x: len(str(x)))
dat['diff_len'] = dat.len_q1 - dat.len_q2
dat['len_char_q1'] = dat.question1.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
dat['len_char_q2'] = dat.question2.apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
dat['len_word_q1'] = dat.question1.apply(lambda x: len(str(x).split()))
dat['len_word_q2'] = dat.question2.apply(lambda x: len(str(x).split()))
dat['common_words'] = dat.apply(lambda x: len(set(str(x['question1']).lower().split()).intersection(set(str(x['question2']).lower().split()))), axis=1)
dat['fuzz_qratio'] = dat.apply(lambda x: fuzz.QRatio(str(x['question1']), str(x['question2'])), axis=1)
dat['fuzz_WRatio'] = dat.apply(lambda x: fuzz.WRatio(str(x['question1']), str(x['question2'])), axis=1)
dat['fuzz_partial_ratio'] = dat.apply(lambda x: fuzz.partial_ratio(str(x['question1']), str(x['question2'])), axis=1)
dat['fuzz_token_sort_ratio'] = dat.apply(lambda x: fuzz.token_sort_ratio(str(x['question1']), str(x['question2'])), axis=1)
dat.head()

,question1,question2,len_q1,len_q2,diff_len,len_char_q1,len_char_q2,len_word_q1,len_word_q2,common_words,fuzz_qratio,fuzz_WRatio,fuzz_partial_ratio,fuzz_token_sort_ratio
0,what is the time rigt now in india?,what time is it in pakistan?,35,28,7,14,12,8,6,4,59,67,56,62


In [54]:
x=dat[fe]

In [55]:
pred=clf.predict(x)

C:\Users\ACER\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [57]:
if pred[0]==1:
    print("The questions are similar")
else :
    print("The questions are different")

The questions are different
